In [15]:
import pandas as pd
import re
from glob import glob
from tqdm.notebook import tqdm

In [2]:
files = glob('./data/*.csv')

In [3]:
result = pd.DataFrame(columns=['title','contents'])
for ifile in files:
    df = pd.read_csv(ifile)
    result = pd.concat([result,df], axis=0)

In [4]:
result.head()

,title,contents
0,\n\t\t\t\t정부조직법\n\t\t\t\t\n\t\t\t,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
1,\n\t\t\t\t학교시설사업 촉진법\n\t\t\t\t\n\t\t\t,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
2,\n\t\t\t\t공공차관의도입및관리에관한법률시행규칙\n\t\t\t\t,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
3,\n\t\t\t\t사법정책자문위원회 규칙\n\t\t\t\t\n\t\t\t,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
4,\n\t\t\t\t연구실 안전환경 조성에 관한 법률 시행규칙\n\t\t\t\t,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...


### 조 별로 나누어서 나이 찾기

#### title strip

In [5]:
result['title'] = result['title'].apply(lambda x : x.strip() )

In [6]:
# index reset
result = result.reset_index(drop=True)

In [8]:
result.head()

,title,contents
0,정부조직법,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
1,학교시설사업 촉진법,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
2,공공차관의도입및관리에관한법률시행규칙,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
3,사법정책자문위원회 규칙,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
4,연구실 안전환경 조성에 관한 법률 시행규칙,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...


#### title 중복 제거

In [11]:
result = result.drop_duplicates(subset='title')
result

,title,contents
0,정부조직법,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
1,학교시설사업 촉진법,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
2,공공차관의도입및관리에관한법률시행규칙,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
3,사법정책자문위원회 규칙,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
4,연구실 안전환경 조성에 관한 법률 시행규칙,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...
...,...,...
5644,교육환경 보호에 관한 법률 시행령,제1조(목적) 이 영은 「교육환경 보호에 관한 법률」에서 위임된 사항과 그 시행에 ...
5645,과학기술정보통신부와 그 소속기관 직제,"제1조(목적) 이 영은 과학기술정보통신부와 그 소속기관의 조직과 직무범위, 그 밖..."
5646,주식ㆍ사채 등의 전자등록에 관한 법률 시행령,제1조(목적) 이 영은 「주식ㆍ사채 등의 전자등록에 관한 법률」에서 위임된 사항과 ...
5647,휴직법관에 대한 보수지급에 관한 규칙,제1조(목적 및 적용범위) 이 규칙은 「법원조직법」 제51조제2항의 규정에 의한 휴...


In [12]:
result = result.set_index('title')
result.head()

,contents
title,
정부조직법,제1조(목적) 이 법은 국가행정사무의 체계적이고 능률적인 수행을 위하여 국가행정기...
학교시설사업 촉진법,제1조(목적) 이 법은 초등학교ㆍ중학교ㆍ고등학교 및 특수학교의 시설의 설치ㆍ이전 및...
공공차관의도입및관리에관한법률시행규칙,제1조(목적) 이 규칙은 공공차관의도입및관리에관한법률 및 동법시행령에서 위임된 사항...
사법정책자문위원회 규칙,제1조 (목적) 이 규칙은 법원조직법 제25조에 규정된 사법정책자문위원회(이하 ‘위...
연구실 안전환경 조성에 관한 법률 시행규칙,제1조(목적) 이 규칙은 「연구실 안전환경 조성에 관한 법률」 및 같은 법 시행령에...


In [13]:
result.describe()

,contents
count,5033
unique,5033
top,제1조(목적) 이 영은 「인터넷주소자원에 관한 법률」에서 위임된 사항과 그 시행에 ...
freq,1


In [19]:
result.isnull().sum()

contents    25
dtype: int64

In [20]:
result = result.dropna() # 결측치 제거

In [21]:
len(result)

5033

#### 나이 후보군 찾기

In [187]:
result_df = pd.DataFrame()
cnt = 0
for law in tqdm(result.index): # 법령명 하나의
    for text in result.loc[law]['contents'].split('.'): # 문장 . 으로 분리 후
        
        age_find = re.findall(r'[만]*[\s]*[0-9]+[\s]*세[^대환제트관임월][\s]*[이]?[상하전후내외]?[미]?[만]?[초]?[과]?[까]?[지]?[부]?[터]?', text)
        if age_find:
            jo_text = re.findall(r'제[0-9]+조[의]?[0-9]?\([가-힇\s]+\)', text)
            
            if jo_text: # 같은 조에 속할때
                title = law
                jo_title = jo_text[0]
                
                age_list = [re.sub('[^0-9만세이상하전후내외미만초과까지부터]+', '', i) for i in age_find] # 한 줄 안에 여러 나이가 있을때
                jo_contents = re.sub('[\[\]\>]', '', text) 
                result_df = result_df.append({
                    'title':title,
                    'jo_title':jo_title,
                    'jo_contents': jo_contents,
                    'age_list': age_list
                }, ignore_index=True)
            else: # 조 안에서 여러 줄로 나누어 져 있을때
                jo_title = jo_text[0]
                jo_contents = re.sub('[\[\]\>]', '', text) 
                title = law
                age_list = [re.sub('[^A-Za-z0-9가-힣]', '', i) for i in age_find] # 한 줄 안에 여러 나이가 있을때
                result_df = result_df.append({
                    'title':title,
                    'jo_title':jo_title,
                    'jo_contents': jo_contents,
                    'age_list': age_list
                }, ignore_index=True)


IndexError: list index out of range

In [188]:
result_df

,age_list,jo_contents,jo_title,title
0,[13세이상],"제18조의2(자의 의견의 청취) 가정법원이 미성년자인 자의 친권자 지정, 양육과 ...",제18조의2(자의 의견의 청취),가사소송규칙
1,[13세이상],제59조의2(관계자의 의견의 청취) ① 가정법원은 「민법」 제781조제5항의 규정...,제59조의2(관계자의 의견의 청취),가사소송규칙


In [174]:
def age_norm(d):
    d = d['age_list']
    for j in d :
        if len(j.split('세'))==2 and j.split('세')[-1]!='': 
            if j.split('세')[-1] not in ['이상','이하','이후','이전','초과','미만','까지', '미만까지','전까지']:
                if j.startswith('만'):
                    return pd.Series((True,int(j.split('세')[0].split('만')[-1]), False,False))
                return pd.Series((False,int(j.split('세')[0]), False,False))
            else:
                if j.split('세')[-1] in ['이후','초과', '이상', '부터']:
                    if j.startswith('만'):
                        return pd.Series((True,int(j.split('세')[0].split('만')[-1]), True,False))
                    return pd.Series((False,int(j.split('세')[0]), True,False))
                else:
                    if j.startswith('만'):
                        return pd.Series((True, int(j.split('세')[0].split('만')[-1]), False,True))
                    return pd.Series((False,int(j.split('세')[0]), True,False))
        else:
            if j.startswith('만'):
                return pd.Series((True,int(j.split('세')[0].split('만')[-1]), False,False))
            return pd.Series((False,int(j.split('세')[0]), False,False))

In [175]:
result_df[['man_TF','age','gt','lt']] = result_df.apply(age_norm ,axis=1)

In [176]:
result_df.describe()

,age
count,1389.000000
mean,32.113751
std,57.396907
min,1.000000
25%,17.000000
50%,19.000000
75%,60.000000
max,2015.000000


In [177]:
result_df = result_df[result_df['age']<100] # 100 이상의 조문들은 삭제

In [178]:
result_df.describe()

,age
count,1388.000000
mean,30.685159
std,21.446659
min,1.000000
25%,17.000000
50%,19.000000
75%,60.000000
max,80.000000


In [181]:
result_df[result_df['age']<40].sample(10)

,age_list,jo_contents,jo_title,title,man_TF,age,gt,lt
1181,"[22세, 23세, 24세]",전문대학 및 「평생교육법」 제31조제4항에 따른 전공대학(이하 “전공대학”이라 한...,제7조(병역준비역 편입자 조사),병역법 시행령,False,22,False,False
1076,[18세이상],고등학교 졸업학력 검정고시는 18세 이상으로 고등학교 교육과정을 이수 중인 사람전...,제12조(보호처분 취소대상자 통지),보호소년 등의 처우에 관한 법률 시행령,False,18,True,False
115,[15세미만],15세 미만인 사람을 군대 또는 무장집단에 징집 또는 모병의 방법으로 참여하도록 ...,제10조의3(생계비등의 범위),국제형사재판소 관할 범죄의 처벌 등에 관한 법률,False,15,True,False
1195,"[19세미만, 19세로, 65세이상]","19세 미만인 사람, 19세로서 고등학교 이하의 학교에 재학 중인 사람과 65세 ...",제129조의2(산업체 등에 취업한 사람의 병역의무이행일 연기),병역법 시행령,False,19,True,False
1375,"[19세미만, 19세이상]",본인 및 배우자의 19세 미만의 직계비속 및 19세 이상의 직계비속 중 장애의 정...,제19조(지원절차 등),법관 및 법원공무원수당 등에 관한 규칙,False,19,True,False
1322,[14세미만],제25조(법정대리인의 권리) ①위치정보사업자등이 14세 미만의 아동으로부터 제18...,제25조(법정대리인의 권리),위치정보의 보호 및 이용 등에 관한 법률,False,14,True,False
926,[19세가],대통령령으로 정하는 정도의 장해 상태에 있지 아니한 자녀 또는 손자녀가 19세가 ...,제21조의2(퇴소조치 등),공무원 재해보상법,False,19,False,False
476,"[15세이상, 18세미만]",⑥ 제3항은 15세 이상 18세 미만의 근로자에 대하여는 적용하지 아니한다,제42조(주민등록증 분실신고 등),근로기준법,False,15,True,False
424,[18세미만],18세 미만의 사람 2,제18조의5(공청회의 개최),원자력안전법,False,18,True,False
1359,[30세이하],다음 각 목의 어느 하나에 해당하는 30세 이하인 사람,제38조의2(공동생활시설의 이용지원 등),병역법 시행규칙,False,30,True,False


In [184]:
result_df[result_df['title']=='5ㆍ18민주유공자예우에 관한 법률']

,age_list,jo_contents,jo_title,title,man_TF,age,gt,lt
205,[60세미만],60세 미만의 직계존속(直系尊屬)과 성년인 형제자매가 없는 미성년 제매(弟妹) ...,제111조(승객 및 승무원의 좌석 등),5ㆍ18민주유공자예우에 관한 법률,False,60,True,False
206,"[60세미만, 60세미만]",「병역법」 제2조제1항제17호의2에 따른 대체복무요원으로 소집된 사람 ⑥ 제1항...,제111조(승객 및 승무원의 좌석 등),5ㆍ18민주유공자예우에 관한 법률,False,60,True,False
207,[30세이전],5ㆍ18민주화운동사망자 또는 행방불명자의 자녀 및 미성년 제매와 제1호에 해당하는...,제111조(승객 및 승무원의 좌석 등),5ㆍ18민주유공자예우에 관한 법률,False,30,True,False
208,"[65세이상, 60세이상, 60세이상, 55세이상]",제55조(양로지원) 5ㆍ18민주유공자 또는 그 유족(자녀는 제외한다)으로서 65...,제55조(양로지원),5ㆍ18민주유공자예우에 관한 법률,False,65,True,False
209,"[19세가, 19세가]","다만, 양육지원을 받고 있는 사람으로서 19세가 된 사람이 고등학교ㆍ대학 또는 이...",제55조(양로지원),5ㆍ18민주유공자예우에 관한 법률,False,19,False,False


In [126]:
exclude_list = ['장애인고용촉진 및 직업재활법','경찰대학의 학사운영에 관한 규정']

In [127]:
result_df['title'].unique()

array(['가사소송규칙', '전통시장 및 상점가 육성을 위한 특별법 시행령',
       '자동차 및 자동차부품의 성능과 기준에 관한 규칙', '우편법 시행규칙', '지방공무원 임용령', '소득세법',
       '빈집 및 소규모주택 정비에 관한 특례법', '가족관계의 등록 등에 관한 법률', '주민등록법 시행규칙',
       '총포ㆍ도검ㆍ화약류 등의 안전관리에 관한 법률', '주택공급에 관한 규칙', '산림보호법 시행령',
       '해양경찰청 소속 경찰공무원 임용에 관한 규정', '고등교육법', '위해성 경찰장비의 사용기준 등에 관한 규정',
       '고용보험법', '선원법 시행령', '국군간호사관학교 설치법', '아동수당법',
       '해양생태계의 보전 및 관리에 관한 법률 시행규칙', '초ㆍ중등교육법 시행규칙', '공무원임용령', '감사원법',
       '여권법 시행령', '의약품 등의 안전에 관한 규칙', '노인복지법 시행규칙',
       '고용보험 및 산업재해보상보험의 보험료징수 등에 관한 법률',
       '군용비행장ㆍ군사격장 소음 방지 및 피해 보상에 관한 법률 시행령', '의료법 시행규칙', '개인정보 보호법 시행령',
       '선박직원법', '여객자동차 운수사업법', '제대군인지원에 관한 법률 시행령', '전문경력관 규칙', '건설기계관리법',
       '수산업ㆍ어촌 공익기능 증진을 위한 직접지불제도 운영에 관한 법률', '사립학교법 시행령',
       '국제형사재판소 관할 범죄의 처벌 등에 관한 법률', '공연법', '도로교통법 시행규칙', '국적법 시행규칙',
       '국민건강증진법 시행령', '도로교통법', '재외동포의 출입국과 법적 지위에 관한 법률', '보험업법 시행령',
       '국가보훈대상자 의료지원에 관한 규칙', '민법', '교육공무원법', '아이돌봄 지원법 시행규칙', '공직선거법',
       '사행행위 등 규제 및 처벌 특례법 시행규칙', '노인복지법', '여권

In [ ]:
def age_hist(d):
    
    age_list = d['age']
    result = [0 for j in range(11)]
    for j in [int(iage.split('세')[0]) for iage in age_list if not iage.split('세')[0].startswith('만') ]:
        jindex = int(j/10)

        result[jindex]+=1
    for j in [int(iage.split('세')[0].split('만')[-1]) for iage in age_list if iage.split('세')[0].startswith('만') ]:
        jindex = int(j/10)
        result[jindex]+=1
    return pd.Series(result)

In [ ]:
df[["{}대".format(i) for i in range(0,110,10)]] = df.apply(age_hist, axis=1)

단순 연령대별 시각화

In [ ]:
tmp_df = df[["{}대".format(i) for i in range(0,110,10)]]

In [ ]:
tmp_df

In [ ]:
import numpy as np

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rc('font', family='Malgun Gothic')
plt.rcParams["figure.figsize"] = (5,5)

In [ ]:
import seaborn as sns

In [ ]:
tmp_df.sum().to_frame()

In [ ]:
tmp_df

In [ ]:
# tmp = tmp_df.sum()
sns.heatmap(tmp_df.sum().to_frame().T,cmap='YlGnBu' )
plt.show()


In [ ]:
for i in range(0, len(tmp_df[tmp_df['0대']!=0].sort_values(by='0대', ascending=False)),50):
    tmp = tmp_df[tmp_df['0대']!=0].sort_values(by='0대', ascending=False).iloc[i:i+50]
    sns.heatmap(tmp,cmap='YlGnBu' )
    plt.show()

In [ ]:
for i in range(0, len(tmp_df[tmp_df['10대']!=0].sort_values(by='10대', ascending=False)),20):
    tmp = tmp_df[tmp_df['10대']!=0].sort_values(by='10대', ascending=False).iloc[i:i+20]
    sns.heatmap(tmp,cmap='YlGnBu' )
    plt.show()

In [ ]:
for i in range(0, len(tmp_df[tmp_df['60대']!=0].sort_values(by='60대', ascending=False)),20):
    tmp = tmp_df[tmp_df['60대']!=0].sort_values(by='60대', ascending=False).iloc[i:i+20]
    sns.heatmap(tmp,cmap='YlGnBu' )
    plt.show()

In [ ]:
df

In [ ]:
age_dict = {}
for idx, row in df.iterrows():
    data = row['age']
    for item in data:
        if item not in age_dict.keys():
            age_dict[item] = 1
        else:
            age_dict[item]+=1

In [ ]:
age_dict